In [1]:
%load_ext tensorboard

In [2]:
import os

import tensorflow as tf
import datetime
import time

from midi_utils import *
import numpy as np

import matplotlib.pyplot as plt

In [3]:
# all_samples = []

# for dirpath, _, filenames in os.walk('OnlyPiano_TPB128'):
#     for File in filenames:
#         path = os.path.join(dirpath, File)
#         mid = mido.MidiFile(path)
#         try:
#             samples = midi_to_pianoroll(mid)
#             all_samples.append(samples)
#         except:
#             pass
# all_samples = np.array(all_samples, dtype=np.uint8)
# np.save('samples.npy', all_samples)

In [3]:
x = np.load('samples.npy')

In [4]:
X_train = x[:-28]
X_valid = x[-28:]

In [5]:
def rounded_accuracy(y_true, y_pred):
    return tf.keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

In [7]:
X_train.shape

(189, 128, 512)

In [8]:
np.random.seed(42)


def create_model():
    stacked_encoder = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=[128, 512]),
        tf.keras.layers.Dense(1024, activation="selu"),
        tf.keras.layers.Dense(512, activation="selu"),
    ])
    stacked_decoder = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1024, activation="selu", input_shape=[512]),
        tf.keras.layers.Dense(128 * 512, activation="sigmoid"),
        tf.keras.layers.Reshape([128, 512])
    ])
    stacked_ae = tf.keras.models.Sequential([stacked_encoder, stacked_decoder])

    stacked_ae.compile(loss="binary_crossentropy",
                       optimizer=tf.keras.optimizers.SGD(learning_rate=1.5),
                       metrics=[rounded_accuracy])

    return stacked_ae

## Primeiro exemplo de Checkpoints

In [10]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    save_weights_only=True,
    verbose=1,
)

model = create_model()
history = model.fit(
    X_train,
    X_train,
    epochs=10,
    validation_data=(X_valid, X_valid),
    callbacks=[cp_callback],
)

Epoch 1/10
6/6 [==============================] - 4s 330ms/step - loss: -12.7977 - rounded_accuracy: 0.5012 - val_loss: -5.8872 - val_rounded_accuracy: 0.4870

Epoch 00001: saving model to training_1\cp.ckpt


UnknownError: Failed to rename: training_1\cp.ckpt_temp/part-00000-of-00001.data-00000-of-00001.tempstate392404955375300871 to: training_1\cp.ckpt_temp/part-00000-of-00001.data-00000-of-00001 : Acesso negado.
; Input/output error [Op:SaveV2]

In [9]:
model = create_model()
model.load_weights(checkpoint_path)

## Segundo exemplo de checkpoints

In [10]:
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 period=5)

model = create_model()
history = model.fit(X_train,
                    X_train,
                    epochs=20,
                    validation_data=(X_valid, X_valid),
                    callbacks=[cp_callback],
                    verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt


In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0020.ckpt'

## Terceiro Exemplo de checkpoints

In [12]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

In [13]:
# unique, counts = np.unique(a, return_counts=True)
# dict(zip(unique, counts))

In [14]:
predict = model.predict(X_valid)

for i in range(10):
    mid = pianoroll_to_midi(predict[i])
    mid.save(f'generatedSongs/music{i+1}.mid')

## Tensorboard

In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices(X_train)
valid_dataset = tf.data.Dataset.from_tensor_slices(X_valid)

In [18]:
loss_object = tf.keras.losses.binary_crossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=1.5)

In [19]:
train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')
test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [20]:
def train_step(model, optimizer, x_train):
    with tf.GradientTape() as tape:
        predictions = model(x_train, training=True)
        loss = loss_object(predictions)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)


def test_step(model, x_test, y_test):
    predictions = model(x_test)
    loss = loss_object(y_test, predictions)

    test_loss(loss)
    test_accuracy(y_test, predictions)

In [21]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

In [22]:
model = create_model()  # reset our model

EPOCHS = 5

for epoch in range(EPOCHS):
    for (x_train) in train_dataset:
        train_step(model, optimizer, x_train)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    for (x_test, y_test) in test_dataset:
        test_step(model, x_test, y_test)
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(
        template.format(epoch + 1, train_loss.result(),
                        train_accuracy.result() * 100, test_loss.result(),
                        test_accuracy.result() * 100))

    # Reset metrics every epoch
    train_loss.reset_states()
    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

In [23]:
%tensorboard --logdir logs/fit